In [1]:
import pandas as pd
from IPython.core import display as ICD
from multiprocessing import Pool
import numpy as np
from datetime import timedelta
import re
import os
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
import plotly
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from itertools import groupby, product
import json
le = LabelEncoder()

In [2]:
def read_lcm_output(input_name,folder="lcm_results"):
    """Read and restructure LCM output file,rename columns output a df """
    file = f'{folder}/{input_name}'
    df = pd.read_csv(file,header=None)
    df.columns = ["user_ids","support","itemsets","period","property_values"]
    df["period"] = pd.to_datetime(df["period"])
    df["user_ids"] = df.user_ids.apply(lambda x : np.array([int(z) for z in x.split(" ") if z != ""]))
    return df


def read_lcm_output_total(input_name,folder="../lcm_results"):
    """Read and restructure LCM output file,rename columns output a df """
    file = f'{folder}/{input_name}'
    df = pd.read_csv(file,header=None)
    df.columns = ["user_ids","support","itemsets","period","property_values"]
    df["period"] = pd.to_datetime(df["period"])
    df["user_ids"] = df.user_ids.apply(lambda x : [int(z.replace('"',"")) for z in x[1:-1].split(",") if z != ""])
    return df


# Data preprocessing for echarts 

In [14]:
def get_articles_descriptions(x,items,encoder):
    if isinstance(x,int):
        x = "[x]
    ee = encoder.inverse_transform([int(i) for i in x.split()])
    return items.loc[items.index.isin(ee)].DESCRIPTION.tolist()
def format_links(x):
    res = []
    for i in x[0]:
        for idx in range(len(i)-1):
            res.append([i[idx],i[idx+1],x["index"]])
    return res

def make_links(e,index):
    """Product of groups ids for each two consecutive groups periods"""
    prev = e[0]
    for i in e[1:]:
        yield from product(prev,i,[index])
        prev = i

def extract_demographics(input_file):
    demographics = re.findall("\[([A-Z|a-z|_]+),?([A-Z|a-z|_]+)?\]",input_file)[0]
    return  [i for i in demographics if i !=""]
    
def sankey_preprocessing(input_file,stats_folder='../plots/stats',encoders_folder="../plots/encoders",links_folder="../plots/links",groups_folder='../plots/groups',users_demographics = ["DEPARTEMENT","SEX","AGE"],groups_demographics=["STATION_MGT_TYPE","DEPARTEMENT"],user_apparition_threshold=0,user_nunique_periods_threshold=3,keep_all_groups_in_periods=[]):

    demographics = extract_demographics(input_file)
    users = pd.read_csv("../datasets/Total/users.csv",sep=";")
    df = read_lcm_output_total(input_file).sort_values("period").reset_index(drop=True)
    
    file = f'../plots/links/{input_file}'
    mlb = MultiLabelBinarizer(sparse_output=True)
    _df = mlb.fit_transform(df.user_ids.tolist()).astype(bool)
    _df = pd.DataFrame(_df.toarray(),columns=mlb.classes_)
    
    e = _df.sum()
    _df = _df[e[e>user_apparition_threshold].index] 
    _df  = _df.T.apply(lambda x : np.where(x)[0],axis=1)
    e = _df.to_frame()[0].apply(lambda x: list(list(z) for idx,z in groupby(x,lambda y: df.iloc[y].period)))
    e = e[e.apply(lambda x:len(x))>user_nunique_periods_threshold]
    
    res = []
    e.to_frame().reset_index().apply(lambda x: [res.append(i) for i in make_links(x[0],x["index"])],axis=1)
    links = pd.DataFrame(res)
  

    links.columns = ["source","target","user_id"]
    links.groupby(["source","target"])["user_id"].apply(lambda x: ','.join(str(i) for i in x)).to_frame().to_csv(file)

#     # Users demographics stats  
#     file = f'../plots/stats/users/{input_file}'
#     stats = {}
#     users_stats = links[["user_id"]].drop_duplicates().merge(users,left_on="user_id",right_on="CUST_ID")[users.columns]
#     for i in users_demographics:
#         b = users_stats.groupby(i).apply(lambda x: {"name":x[i].unique()[0],"value":x.CUST_ID.shape[0],"users":",".join(str(i) for i in x.CUST_ID)}).values
#         stats[i] = b.tolist()
        
#     with open(file, 'w') as outfile:
#         json.dump(stats, outfile)
  
    
#     # Users
#     file = f'../plots/users/{input_file}'
#     users_stats.to_csv(file)
    
    # filter groups to the ones appearing in the links
    file = f'{groups_folder}/{input_file}'
    groups_to_keep = np.unique(np.union1d(links.source.unique(),links.target.unique()))

    groups_to_keep = np.union1d(groups_to_keep,df[df.period.isin(keep_all_groups_in_periods)].index)
    df_reduced_filtred = df.loc[groups_to_keep].dropna()
    
    df_reduced_filtred['depth'] = le.fit_transform(df_reduced_filtred.period)/df_reduced_filtred.period.nunique()
    df_reduced_filtred['size'] = df_reduced_filtred.user_ids.apply(lambda x : len(x))
    if len(demographics)==1:
        df_reduced_filtred[demographics[0]]= df_reduced_filtred.property_values
    else:
        df_reduced_filtred[demographics]= df_reduced_filtred.property_values.str.split("_",expand=True)
    
    # Encoding items to their initial ID + adding names
    items = pd.read_csv("../datasets/Total/items.csv")
    encoder = LabelEncoder()
    encoder.classes_ = np.load(f'{encoders_folder}/15D-3-[2-5000]-[AGE]-lcm.out.npy')
    items = items.set_index("ARTICLE_ID")
    df_reduced_filtred["itemset_name"] = df_reduced_filtred["itemsets"].apply(lambda x : get_articles_descriptions(x,items,encoder))
    df_reduced_filtred.to_csv(file)
    
    # Groups demographics stats 
    file = f'../plots/stats/groups/{input_file}'
    stats = {}
    for i in np.intersect1d(groups_demographics,demographics):
        b = df_reduced_filtred.groupby(i).apply(lambda x : {"name":x[i].unique()[0],"value":x.index.shape[0],"groups":",".join(str(i) for i in x.index)}).values
        stats[i]=str(b.tolist())
    with open(file, 'w') as outfile:
        json.dump(stats, outfile)

    print("Done",input_file)


In [15]:
input_file = "2M-20-[1-None]-[a]-lcm.out"
for i in [input_file]:
    print(i,extract_demographics(i))
    e = sankey_preprocessing(i,keep_all_groups_in_periods=[])


2M-20-[1-None]-[a]-lcm.out ['a']
4125


AttributeError: 'int' object has no attribute 'split'

In [9]:
sankey_preprocessing("M-2-[1-None]-[SEX]-lcm.out",user_nunique_periods_threshold=1)

Done M-2-[1-None]-[SEX]-lcm.out


### Draft

In [11]:
input_file ="M-2-[1-None]-[SEX]-lcm.out"
df = read_lcm_output_total(input_file).sort_values("period").reset_index(drop=True)
df.user_ids

0       [114621, 123927, 139931, 147495, 154330, 16898...
1                       [419062, 448392, 552027, 1058591]
2               [419062, 448392, 552027, 562198, 1058591]
3       [199673, 247735, 277744, 453671, 555714, 59744...
4                     [522344, 1036452, 1045705, 1082751]
                              ...                        
1060             [111931, 172631, 218865, 268731, 275156]
1061             [267234, 268731, 443377, 488580, 568735]
1062    [140231, 194407, 210460, 242618, 287308, 28773...
1063     [336312, 337630, 338619, 374604, 404236, 752568]
1064                     [209988, 524542, 753456, 901254]
Name: user_ids, Length: 1065, dtype: object

### sankey preprocessing updating new file instead of creating from scratch

In [12]:
def sankey_preprocessing(old_output,input_file,stats_folder='../plots/stats',encoders_folder="../plots/encoders",links_folder="../plots/links",groups_folder='../plots/groups',users_demographics = ["DEPARTEMENT","SEX","AGE"],groups_demographics=["STATION_MGT_TYPE","DEPARTEMENT"],user_apparition_threshold=0,user_nunique_periods_threshold=3,keep_all_groups_in_periods=[]):

    demographics = extract_demographics(input_file)
    users = pd.read_csv("../datasets/Total/users.csv",sep=";")
    df = read_lcm_output_total(input_file).sort_values("period").reset_index(drop=True)
    
    file = f'../plots/links/{input_file}'
    mlb = MultiLabelBinarizer(sparse_output=True)
    _df = mlb.fit_transform(df.user_ids.tolist()).astype(bool)
    _df = pd.DataFrame(_df.toarray(),columns=mlb.classes_)
    
    e = _df.sum()
    _df = _df[e[e>user_apparition_threshold].index] 
    _df  = _df.T.apply(lambda x : np.where(x)[0],axis=1)
    e = _df.to_frame()[0].apply(lambda x: list(list(z) for idx,z in groupby(x,lambda y: df.iloc[y].period)))
    e = e[e.apply(lambda x:len(x))>user_nunique_periods_threshold]
    
    res = []
    e.to_frame().reset_index().apply(lambda x: [res.append(i) for i in make_links(x[0],x["index"])],axis=1)
    links = pd.DataFrame(res)
  

    links.columns = ["source","target","user_id"]
    links.groupby(["source","target"])["user_id"].apply(lambda x: ','.join(str(i) for i in x)).to_frame().to_csv(file)

    # filter groups to the ones appearing in the links
    file = f'{groups_folder}/{input_file}'
    groups_to_keep = np.unique(np.union1d(links.source.unique(),links.target.unique()))

    groups_to_keep = np.union1d(groups_to_keep,df[df.period.isin(keep_all_groups_in_periods)].index)
    df_reduced_filtred = df.loc[groups_to_keep].dropna()
    
    df_reduced_filtred['depth'] = le.fit_transform(df_reduced_filtred.period)/df_reduced_filtred.period.nunique()
    df_reduced_filtred['size'] = df_reduced_filtred.user_ids.apply(lambda x : len(x))
    if len(demographics)==1:
        df_reduced_filtred[demographics[0]]= df_reduced_filtred.property_values
    else:
        df_reduced_filtred[demographics]= df_reduced_filtred.property_values.str.split("_",expand=True)
    
    # Encoding items to their initial ID + adding names
    items = pd.read_csv("../datasets/Total/items.csv")
    encoder = LabelEncoder()
    encoder.classes_ = np.load(f'{encoders_folder}/15D-3-[2-5000]-[AGE]-lcm.out.npy')
    items = items.set_index("ARTICLE_ID")
    df_reduced_filtred["itemset_name"] = df_reduced_filtred["itemsets"].apply(lambda x : get_articles_descriptions(x,items,encoder))
    df_reduced_filtred.to_csv(file)
    
#     # Groups demographics stats 
#     file = f'../plots/stats/groups/{input_file}'
#     stats = {}
#     for i in np.intersect1d(groups_demographics,demographics):
#         b = df_reduced_filtred.groupby(i).apply(lambda x : {"name":x[i].unique()[0],"value":x.index.shape[0],"groups":",".join(str(i) for i in x.index)}).values
#         stats[i]=str(b.tolist())
#     with open(file, 'w') as outfile:
#         json.dump(stats, outfile)

    print("Done",input_file)


In [14]:
df = read_lcm_output_total(output_file).sort_values("period").reset_index(drop=True)
df

NameError: name 'output_file' is not defined